In [2]:
import pandas as pd
pd.set_option('display.max_columns', None)
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import (
    PowerTransformer,
    MinMaxScaler,
    OneHotEncoder,
    OrdinalEncoder,
    StandardScaler
)
from sklearn.tree import DecisionTreeRegressor
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, root_mean_squared_error

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV


In [3]:
data = pd.read_csv('preprocessed_data.csv')
display(data.head())
print(data.shape)

,numerical__num_pipeline__serviceCharge,numerical__num_pipeline__pricetrend,numerical__num_pipeline__baseRent,numerical__num_pipeline__livingSpace,numerical__num_pipeline__totalRentPerSquareMeter,categorical__binary__newlyConst,categorical__binary__balcony,categorical__binary__hasKitchen,categorical__binary__cellar,categorical__binary__lift,categorical__binary__garden,categorical__low_card__stateName_Baden_Württemberg,categorical__low_card__stateName_Bayern,categorical__low_card__stateName_Berlin,categorical__low_card__stateName_Brandenburg,categorical__low_card__stateName_Bremen,categorical__low_card__stateName_Hamburg,categorical__low_card__stateName_Hessen,categorical__low_card__stateName_Mecklenburg_Vorpommern,categorical__low_card__stateName_Niedersachsen,categorical__low_card__stateName_Nordrhein_Westfalen,categorical__low_card__stateName_Rheinland_Pfalz,categorical__low_card__stateName_Saarland,categorical__low_card__stateName_Sachsen,categorical__low_card__stateName_Sachsen_Anhalt,categorical__low_card__stateName_Schleswig_Holstein,categorical__low_card__stateName_Thüringen,categorical__low_card__telekomTvOffer_ONE_YEAR_FREE,categorical__low_card__telekomTvOffer_ON_DEMAND,categorical__low_card__telekomTvOffer_unknown,categorical__low_card__petsAllowed_negotiable,categorical__low_card__petsAllowed_no,categorical__low_card__petsAllowed_unknown,categorical__low_card__petsAllowed_yes,categorical__low_card__typeOfFlat_apartment,categorical__low_card__typeOfFlat_ground_floor,categorical__low_card__typeOfFlat_half_basement,categorical__low_card__typeOfFlat_loft,categorical__low_card__typeOfFlat_maisonette,categorical__low_card__typeOfFlat_other,categorical__low_card__typeOfFlat_penthouse,categorical__low_card__typeOfFlat_raised_ground_floor,categorical__low_card__typeOfFlat_roof_storey,categorical__low_card__typeOfFlat_terraced_flat,categorical__med_card__heatingType,categorical__med_card__condition,categorical__med_card__interiorQual,categorical__med_card__address,categorical__high_card__geo_plz,categorical__high_card__cityName,categorical__high_card__cityDistrict,categorical__high_card__firingTypes,categorical__high_card__address,categorical__numeric_binned__picturecount,categorical__numeric_binned__noRooms,categorical__ordinal__yearConstructedRange,categorical__ordinal__baseRentRange,categorical__ordinal__livingSpaceRange,categorical__ordinal__yearConstructed_category,categorical__ordinal__floor_category,totalRent
0,0.094884,0.861874,0.736052,0.607771,0.594296,False,False,False,True,False,True,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,747.188398,702.826563,619.734247,951.958199,951.958199,732.622202,1047.432327,812.379328,951.958199,0.0,0.0,1.0,3.0,3.0,1.0,0.0,840.0
1,0.096432,0.826455,0.783812,0.602796,0.660976,True,True,False,True,True,False,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1207.133159,1208.945307,1048.553574,802.044484,800.607042,740.351808,828.959870,805.145414,802.044484,0.0,0.0,8.0,5.0,3.0,3.0,0.0,1300.0
2,0.051460,0.800218,0.679303,0.534208,0.615575,False,True,False,False,False,False,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,768.486728,708.999159,718.760225,584.895738,584.691598,489.436220,587.017336,805.145414,584.895738,0.0,0.0,1.0,1.0,1.0,1.0,0.0,655.0
3,0.074891,0.821065,0.761110,0.605456,0.606967,False,True,False,False,False,False,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,721.002345,635.648127,718.760225,981.975714,981.975714,851.943960,994.498427,782.735822,981.975714,0.0,0.0,0.0,4.0,3.0,0.0,0.0,903.0
4,0.075794,0.859457,0.670370,0.518733,0.627908,False,True,True,False,False,False,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,

(20596, 61)


In [4]:
(
    data
    .isna()
    .sum()
    .sort_values(ascending=False)
)

,0
totalRent,1
categorical__high_card__cityDistrict,1
categorical__low_card__typeOfFlat_raised_ground_floor,1
categorical__low_card__typeOfFlat_roof_storey,1
categorical__low_card__typeOfFlat_terraced_flat,1
...,...
categorical__low_card__stateName_Schleswig_Holstein,0
categorical__low_card__stateName_Thüringen,0
categorical__low_card__telekomTvOffer_ONE_YEAR_FREE,0
categorical__low_card__telekomTvOffer_ON_DEMAND,0


In [5]:
print(data.shape)
data.dropna(inplace=True)
print(data.shape)

(20596, 61)
(20595, 61)


In [6]:
X = data.drop(['totalRent','numerical__num_pipeline__totalRentPerSquareMeter'], axis =1)
y = data['totalRent']

In [7]:
X_full_train, X_test, y_full_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_full_train, y_full_train, test_size=0.25, random_state=42)

print(f"X_train shape: {X_train.shape}")
print(f"X_val shape: {X_val.shape}")
print(f"X_test shape: {X_test.shape}")


X_train shape: (12357, 59)
X_val shape: (4119, 59)
X_test shape: (4119, 59)


In [8]:
from sklearn.metrics import r2_score, mean_squared_error

def adjusted_r2_score(r2, n, p):
    """
    Calculate the adjusted R² score.

    Parameters:
    - r2: R² score
    - n: Number of observations
    - p: Number of predictors/features

    Returns:
    - Adjusted R² score
    """
    return 1 - ((1 - r2) * (n - 1)) / (n - p - 1)


### Decison Tree Regressor

In [11]:
from sklearn.tree import DecisionTreeRegressor

# Intialize the DecisionTreeRegressor
dt = DecisionTreeRegressor(random_state=42)

# Use RandomizedSearchCV for a quicker search
param_grid = {
    'max_depth': [None, 5, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['sqrt', 'log2', None],
    'splitter': ['best', 'random']
}

# Use RandomizedSearchCV for a quicker search
random_search = RandomizedSearchCV(
    estimator=dt,
    param_distributions=param_grid,
    scoring='neg_root_mean_squared_error',  # Use RMSE as evaluation metric
    cv=5,  # 5-fold cross-validation
    verbose=2,
    random_state=42,
    n_jobs=-1  # Use all available processors
)

# Fit the search on training data
random_search.fit(X_train, y_train)

# Best parameters and score
print("Best Hyperparameters:", random_search.best_params_)
print("\n Best R² Score (Validation):", random_search.best_score_)

best_rf = random_search.best_estimator_

# Evaluate on the validation set using the best model
y_pred_val = best_rf.predict(X_val)
r2_val = r2_score(y_val, y_pred_val)
n_val, p_val = X_val.shape  # Number of rows and features
adjusted_r2_val = adjusted_r2_score(r2_val, n_val, p_val)
rmse_val = np.sqrt(mean_squared_error(y_val, y_pred_val))

print(f"\nValidation Set Performance with Tuned Model:")
print(f"R² score: {r2_val}")
print(f"Adjusted R² score: {adjusted_r2_val}")
print(f"RMSE: {rmse_val}\n")

# Evaluate on the test set using the best model
y_pred_test = best_rf.predict(X_test)
r2_test = r2_score(y_test, y_pred_test)
n_test, p_test = X_test.shape  # Number of rows and features
adjusted_r2_test = adjusted_r2_score(r2_test, n_test, p_test)
rmse_test = np.sqrt(mean_squared_error(y_test, y_pred_test))

print(f"\nTest Set Performance with Tuned Model:")
print(f"R² score: {r2_test}")
print(f"Adjusted R² score: {adjusted_r2_test}")
print(f"RMSE: {rmse_test}")

Fitting 5 folds for each of 10 candidates, totalling 50 fits
Best Hyperparameters: {'splitter': 'best', 'min_samples_split': 5, 'min_samples_leaf': 4, 'max_features': None, 'max_depth': 10}

 Best R² Score (Validation): -250.72149298651465

Validation Set Performance with Tuned Model:
R² score: 0.7720219173927454
Adjusted R² score: 0.7709676215098619
RMSE: 256.2222955870165


Test Set Performance with Tuned Model:
R² score: 0.7530101384111774
Adjusted R² score: 0.7518679216190672
RMSE: 259.8986346375184


### Random Forrest Regressor

In [13]:
# Define the parameter grid
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 5, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['sqrt', 'log2', None]
}

# Initialize the Random Forest Regressor
rf = RandomForestRegressor(random_state=42)

# Use RandomizedSearchCV for a quicker search
random_search = RandomizedSearchCV(
    estimator=rf,
    param_distributions=param_grid,
    scoring='neg_root_mean_squared_error',
    cv=5,
    verbose=2,
    random_state=42,
    n_jobs=-1
)

# Fit the search on training data
random_search.fit(X_train, y_train)

# Best parameters and score
print("Best Hyperparameters:", random_search.best_params_)
print("\n Best R² Score (Validation):", random_search.best_score_)

best_rf = random_search.best_estimator_

# Evaluate on the validation set using the best model
y_pred_val = best_rf.predict(X_val)
r2_val = r2_score(y_val, y_pred_val)
n_val, p_val = X_val.shape  # Number of rows and features
adjusted_r2_val = adjusted_r2_score(r2_val, n_val, p_val)
rmse_val = np.sqrt(mean_squared_error(y_val, y_pred_val))

print(f"\n Validation Set Performance with Tuned Model:")
print(f"R² score: {r2_val}")
print(f"Adjusted R² score: {adjusted_r2_val}")
print(f"RMSE: {rmse_val}\n")

# Evaluate on the test set using the best model
y_pred_test = best_rf.predict(X_test)
r2_test = r2_score(y_test, y_pred_test)
n_test, p_test = X_test.shape  # Number of rows and features
adjusted_r2_test = adjusted_r2_score(r2_test, n_test, p_test)
rmse_test = np.sqrt(mean_squared_error(y_test, y_pred_test))

print(f"\n Test Set Performance with Tuned Model:")
print(f"R² score: {r2_test}")
print(f"Adjusted R² score: {adjusted_r2_test}")
print(f"RMSE: {rmse_test}")

Fitting 5 folds for each of 10 candidates, totalling 50 fits
Best Hyperparameters: {'n_estimators': 100, 'min_samples_split': 2, 'min_samples_leaf': 2, 'max_features': 'sqrt', 'max_depth': None}

 Best R² Score (Validation): -214.70839726147057

 Validation Set Performance with Tuned Model:
R² score: 0.8353032087654741
Adjusted R² score: 0.8345415603344633
RMSE: 217.77746515403496


 Test Set Performance with Tuned Model:
R² score: 0.8348992523466618
Adjusted R² score: 0.8341357357992761
RMSE: 212.49022411202324


In [15]:
# Extract feature weights from the Random Forest model
feature_weights = best_rf.feature_importances_

# Combine feature names with their corresponding weights
feature_weight_pairs = list(zip(X_train.columns, feature_weights))

# Sort the features by their importance in descending order
sorted_features = sorted(feature_weight_pairs, key=lambda x: x[1], reverse=True)

# Print feature weights
print("Feature Weights (sorted by importance):")
for feature, weight in sorted_features:
    print(f"{feature}: {weight:.4f}")

Feature Weights (sorted by importance):
numerical__num_pipeline__baseRent: 0.2452
categorical__ordinal__baseRentRange: 0.1382
numerical__num_pipeline__livingSpace: 0.0902
numerical__num_pipeline__serviceCharge: 0.0867
categorical__ordinal__livingSpaceRange: 0.0738
categorical__high_card__geo_plz: 0.0529
categorical__high_card__address: 0.0491
categorical__med_card__address: 0.0484
categorical__high_card__cityDistrict: 0.0389
categorical__high_card__cityName: 0.0334
numerical__num_pipeline__pricetrend: 0.0229
categorical__med_card__interiorQual: 0.0193
categorical__med_card__condition: 0.0145
categorical__ordinal__yearConstructedRange: 0.0142
categorical__binary__lift: 0.0073
categorical__med_card__heatingType: 0.0062
categorical__high_card__firingTypes: 0.0045
categorical__binary__hasKitchen: 0.0043
categorical__ordinal__yearConstructed_category: 0.0039
categorical__binary__newlyConst: 0.0038
categorical__binary__balcony: 0.0034
categorical__ordinal__floor_category: 0.0030
categorical_

### Linear Regression

In [16]:
# Linear regression
from sklearn.linear_model import LinearRegression
lr = LinearRegression()
lr.fit(X_train, y_train)


# Validation set
y_pred_val = lr.predict(X_val)
r2_val = r2_score(y_val, y_pred_val)
n_val, p_val = X_val.shape  # Number of rows and features
adjusted_r2_val = adjusted_r2_score(r2_val, n_val, p_val)
rmse_val = np.sqrt(mean_squared_error(y_val, y_pred_val))

print(f"Validation Set:")
print(f"R² score: {r2_val}")
print(f"Adjusted R² score: {adjusted_r2_val}")
print(f"RMSE: {rmse_val}\n")

# Test set
y_pred_test = lr.predict(X_test)
r2_test = r2_score(y_test, y_pred_test)
n_test, p_test = X_test.shape  # Number of rows and features
adjusted_r2_test = adjusted_r2_score(r2_test, n_test, p_test)
rmse_test = np.sqrt(mean_squared_error(y_test, y_pred_test))

print(f"Test Set:")
print(f"R² score: {r2_test}")
print(f"Adjusted R² score: {adjusted_r2_test}")
print(f"RMSE: {rmse_test}")



Validation Set:
R² score: 0.7081099886514921
Adjusted R² score: 0.7067601289031333
RMSE: 289.9212084424813

Test Set:
R² score: 0.7016876112292926
Adjusted R² score: 0.7003080508799062
RMSE: 285.6275399971392


In [17]:
# Get feature weights
feature_weights = lr.coef_

# Print feature weights
print("Feature Weights:")
for feature, weight in zip(X_train.columns, feature_weights):
    print(f"{feature}: {weight:.4f}")

Feature Weights:
numerical__num_pipeline__serviceCharge: 2579.5456
numerical__num_pipeline__pricetrend: -551.2043
numerical__num_pipeline__baseRent: 2513.1854
numerical__num_pipeline__livingSpace: 45.8430
categorical__binary__newlyConst: -18.3507
categorical__binary__balcony: -13.4747
categorical__binary__hasKitchen: 1.6916
categorical__binary__cellar: 0.4588
categorical__binary__lift: 29.3053
categorical__binary__garden: -15.4529
categorical__low_card__stateName_Baden_Württemberg: -47.0912
categorical__low_card__stateName_Bayern: 5.3843
categorical__low_card__stateName_Berlin: 19.5159
categorical__low_card__stateName_Brandenburg: 27.8691
categorical__low_card__stateName_Bremen: -38.6443
categorical__low_card__stateName_Hamburg: 65.7146
categorical__low_card__stateName_Hessen: -60.8265
categorical__low_card__stateName_Mecklenburg_Vorpommern: 52.2533
categorical__low_card__stateName_Niedersachsen: -28.6068
categorical__low_card__stateName_Nordrhein_Westfalen: -23.1629
categorical__low_c

### MLP regressor - with RandomSearchCV

In [10]:
from sklearn.neural_network import MLPRegressor

# Define the parameter grid for MLPRegressor
param_grid = {
    'hidden_layer_sizes': [(50,), (100,), (50, 25), (100, 50)],
    'activation': ['relu', 'tanh'],
    'solver': ['adam'],
    'alpha': [0.001, 0.01, 0.1, 1],
    'learning_rate': ['constant', 'adaptive'],
    'max_iter': [1000, 2000, 3000]
}

# Initialize the MLPRegressor
mlp = MLPRegressor(random_state=42)

# Use RandomizedSearchCV for hyperparameter tuning
random_search = RandomizedSearchCV(
    estimator=mlp,
    param_distributions=param_grid,
    scoring='neg_root_mean_squared_error',
    cv=5,
    verbose=2,
    random_state=42,
    n_jobs=-1,
    error_score='raise'  # Raise errors for debugging
)

# Fit the search on training data
random_search.fit(X_train, y_train)

# Best parameters and score
print("Best Hyperparameters:", random_search.best_params_)
print("\nBest R² Score (Validation):", random_search.best_score_)

best_mlp = random_search.best_estimator_

# Evaluate on the validation set using the best model
y_pred_val = best_mlp.predict(X_val)
r2_val = r2_score(y_val, y_pred_val)
rmse_val = np.sqrt(mean_squared_error(y_val, y_pred_val))

print(f"\nValidation Set Performance with Tuned Model:")
print(f"R² score: {r2_val}")
print(f"RMSE: {rmse_val}\n")

# Evaluate on the test set using the best model
y_pred_test = best_mlp.predict(X_test)
r2_test = r2_score(y_test, y_pred_test)
rmse_test = np.sqrt(mean_squared_error(y_test, y_pred_test))

print(f"\nTest Set Performance with Tuned Model:")
print(f"R² score: {r2_test}")
print(f"RMSE: {rmse_test}")

Fitting 5 folds for each of 10 candidates, totalling 50 fits


/usr/local/lib/python3.10/dist-packages/numpy/ma/core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


Best Hyperparameters: {'solver': 'adam', 'max_iter': 1000, 'learning_rate': 'adaptive', 'hidden_layer_sizes': (50, 25), 'alpha': 0.001, 'activation': 'relu'}

Best R² Score (Validation): -264.8067267792527

Validation Set Performance with Tuned Model:
R² score: 0.7294095113241941
RMSE: 262.3241440662395


Test Set Performance with Tuned Model:
R² score: 0.7677809019716394
RMSE: 254.59549479123712


### XGboost

In [ ]:
# !pip install xgboost

In [22]:
import xgboost as xgb
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import r2_score, mean_squared_error
import numpy as np


# Convert binary categorical columns to numeric
binary_columns = ['categorical__binary__cellar', 'categorical__binary__lift', 'categorical__binary__garden']
for col in binary_columns:
    X_train[col] = pd.Categorical(X_train[col]).codes
    X_val[col] = pd.Categorical(X_val[col]).codes
    X_test[col] = pd.Categorical(X_test[col]).codes

# Define the parameter grid for RandomizedSearchCV (for XGBoost)
param_dist = {
    'n_estimators': [100, 200, 500],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 6, 10],
    'subsample': [0.8, 0.9, 1.0],
    'colsample_bytree': [0.8, 0.9, 1.0],
    'alpha': [0, 0.1, 0.5],
    'lambda': [0, 0.1, 1.0],
    'gamma': [0, 0.1, 0.5],
}

# Initialize XGBoost Regressor with enable_categorical=True
xgb_regressor = xgb.XGBRegressor(random_state=42, enable_categorical=True)

# RandomizedSearchCV to find the best hyperparameters
random_search = RandomizedSearchCV(
    xgb_regressor,
    param_distributions=param_dist,
    n_iter=10,
    scoring='neg_mean_squared_error',
    cv=5,
    verbose=1,
    random_state=42
)

# Fit RandomizedSearchCV on the training data
random_search.fit(X_train, y_train)

# Best model from RandomizedSearchCV
best_model = random_search.best_estimator_

# Evaluate the best model on the validation set
y_pred_val = best_model.predict(X_val)
r2_val = r2_score(y_val, y_pred_val)
n_val, p_val = X_val.shape  # Number of rows and features
adjusted_r2_val = adjusted_r2_score(r2_val, n_val, p_val)
rmse_val = np.sqrt(mean_squared_error(y_val, y_pred_val))

print(f"Validation Set with Best XGBoost Regressor:")
print(f"R² score: {r2_val}")
print(f"Adjusted R² score: {adjusted_r2_val}")
print(f"RMSE: {rmse_val}\n")

# Evaluate the best model on the test set
y_pred_test = best_model.predict(X_test)
r2_test = r2_score(y_test, y_pred_test)
n_test, p_test = X_test.shape  # Number of rows and features
adjusted_r2_test = adjusted_r2_score(r2_test, n_test, p_test)
rmse_test = np.sqrt(mean_squared_error(y_test, y_pred_test))

print(f"Test Set with Best XGBoost Regressor:")
print(f"R² score: {r2_test}")
print(f"Adjusted R² score: {adjusted_r2_test}")
print(f"RMSE: {rmse_test}")


Fitting 5 folds for each of 10 candidates, totalling 50 fits
Validation Set with Best XGBoost Regressor:
R² score: 0.831484507776119
Adjusted R² score: 0.8307051995741117
RMSE: 220.28771907593435

Test Set with Best XGBoost Regressor:
R² score: 0.8311445967529534
Adjusted R² score: 0.830363716615661
RMSE: 214.89282526987944


In [ ]:
random_search.best_params_

{'subsample': 1.0,
 'n_estimators': 200,
 'max_depth': 6,
 'learning_rate': 0.1,
 'lambda': 0,
 'gamma': 0.1,
 'colsample_bytree': 0.8,
 'alpha': 0.5}